<a href="https://colab.research.google.com/github/YuriiKlim/AI/blob/learning_without_teacher/HW2_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель



In [1]:
!pip install -q surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [17]:
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import BaselineOnly, accuracy
from surprise import CoClustering
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/ratings.csv")

df

,user_id,movie_id,rating,timestamp
0,172,94969,5.0,1396067836
1,172,98956,4.0,1396067879
2,176,73881,4.0,1499807147
3,221,1900,4.5,1288550866
4,333,33688,4.0,1412015122
...,...,...,...,...
17599,283147,51884,4.0,1180099240
17600,283175,1900,4.5,1201692263
17601,283188,35828,5.0,1448785273
17602,283195,1900,4.5,1138601386


In [3]:
df.describe()

,user_id,movie_id,rating,timestamp
count,17604.000000,17604.000000,17604.000000,1.760400e+04
mean,141251.609009,88913.605942,3.465945,1.403256e+09
std,81731.882435,51285.238375,1.203144,1.515083e+08
min,172.000000,139.000000,0.500000,8.282224e+08
25%,71256.000000,55098.000000,3.000000,1.395695e+09
50%,138487.000000,89554.000000,3.500000,1.456552e+09
75%,210264.500000,134779.500000,4.500000,1.494418e+09
max,283195.000000,193843.000000,5.000000,1.537934e+09


In [18]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(df[["user_id", "movie_id", "rating"]], reader)

In [19]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [20]:
!pip install optuna
import optuna

In [21]:
def objective(trial):
    try:
        algorithm = trial.suggest_categorical('algorithm', ['BaselineOnly', 'SVD', 'KNNBasic', 'CoClustering'])

        if algorithm == 'BaselineOnly':
            method = trial.suggest_categorical('method', ['als', 'sgd'])
            reg_u = trial.suggest_float('reg_u', 1e-5, 1e-1, log=True)
            reg_i = trial.suggest_float('reg_i', 1e-5, 1e-1, log=True)
            bsl_options = {'method': method, 'reg_u': reg_u, 'reg_i': reg_i}
            algo = BaselineOnly(bsl_options=bsl_options)

        elif algorithm == 'SVD':
            n_factors = trial.suggest_int('n_factors', 20, 200)
            n_epochs = trial.suggest_int('n_epochs', 5, 50)
            lr_all = trial.suggest_float('lr_all', 1e-4, 1e-1, log=True)
            reg_all = trial.suggest_float('reg_all', 1e-4, 1e-1, log=True)
            algo = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)

        elif algorithm == 'KNNBasic':
            k = trial.suggest_int('k', 10, 50)
            min_k = trial.suggest_int('min_k', 1, 5)
            sim_options = {
                'name': trial.suggest_categorical('name', ['cosine', 'pearson']),
                'user_based': trial.suggest_categorical('user_based', [True, False])
            }
            algo = KNNBasic(k=k, min_k=min_k, sim_options=sim_options)

        elif algorithm == 'CoClustering':
            n_cltr_u = trial.suggest_int('n_cltr_u', 3, 10)
            n_cltr_i = trial.suggest_int('n_cltr_i', 3, 10)
            n_epochs = trial.suggest_int('n_epochs', 20, 50)
            algo = CoClustering(n_cltr_u=n_cltr_u, n_cltr_i=n_cltr_i, n_epochs=n_epochs)

        algo.fit(trainset)

        predictions = algo.test(testset)
        rmse = accuracy.rmse(predictions, verbose=False)

        return rmse

    except Exception as e:
        print(f"Trial failed with exception: {e}")
        raise optuna.exceptions.TrialPruned()

In [30]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)

[I 2024-09-02 15:39:25,638] A new study created in memory with name: no-name-6c5e11e2-be32-4928-8bd9-35a6ddbe1061
[I 2024-09-02 15:39:27,210] Trial 0 finished with value: 1.0324518581040218 and parameters: {'algorithm': 'CoClustering', 'n_cltr_u': 10, 'n_cltr_i': 6, 'n_epochs': 36}. Best is trial 0 with value: 1.0324518581040218.
[I 2024-09-02 15:39:27,328] Trial 1 finished with value: 0.9594124318030148 and parameters: {'algorithm': 'BaselineOnly', 'method': 'sgd', 'reg_u': 0.00027539894750311557, 'reg_i': 0.003809200632583917}. Best is trial 1 with value: 0.9594124318030148.


Estimating biases using sgd...
Computing the pearson similarity matrix...
Done computing similarity matrix.


[I 2024-09-02 15:39:27,691] Trial 2 finished with value: 1.033698879480597 and parameters: {'algorithm': 'KNNBasic', 'k': 48, 'min_k': 2, 'name': 'pearson', 'user_based': False}. Best is trial 1 with value: 0.9594124318030148.
[I 2024-09-02 15:39:27,751] Trial 3 finished with value: 0.9351975784979176 and parameters: {'algorithm': 'BaselineOnly', 'method': 'als', 'reg_u': 0.00046230099173509917, 'reg_i': 4.255578282366267e-05}. Best is trial 3 with value: 0.9351975784979176.


Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


[I 2024-09-02 15:39:28,022] Trial 4 finished with value: 1.0077960588970545 and parameters: {'algorithm': 'KNNBasic', 'k': 30, 'min_k': 1, 'name': 'cosine', 'user_based': False}. Best is trial 3 with value: 0.9351975784979176.


...

[I 2024-09-02 15:47:20,443] Trial 969 finished with value: 0.9299968221713051 and parameters: {'algorithm': 'SVD', 'n_factors': 71, 'n_epochs': 50, 'lr_all': 0.012336888881054229, 'reg_all': 0.06200975458474044}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:20,893] Trial 970 finished with value: 0.9542422845448314 and parameters: {'algorithm': 'SVD', 'n_factors': 30, 'n_epochs': 48, 'lr_all': 0.002470579877535232, 'reg_all': 0.09982270350960787}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:21,234] Trial 971 finished with value: 0.9239398983649095 and parameters: {'algorithm': 'SVD', 'n_factors': 25, 'n_epochs': 34, 'lr_all': 0.018406422027232233, 'reg_all': 0.08184945225951634}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:21,707] Trial 972 finished with value: 0.9259457079160156 and parameters: {'algorithm': 'SVD', 'n_factors': 36, 'n_epochs': 49, 'lr_all': 0.01565235515829421, 'reg_all': 0.07079498522628852}.

Computing the cosine similarity matrix...


[I 2024-09-02 15:47:26,142] Trial 978 finished with value: 1.1323291172452192 and parameters: {'algorithm': 'KNNBasic', 'k': 19, 'min_k': 2, 'name': 'cosine', 'user_based': True}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:26,727] Trial 979 finished with value: 0.9318866290571344 and parameters: {'algorithm': 'SVD', 'n_factors': 30, 'n_epochs': 50, 'lr_all': 0.02030274536853601, 'reg_all': 0.06392932181646338}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:27,146] Trial 980 finished with value: 0.9198679438614883 and parameters: {'algorithm': 'SVD', 'n_factors': 26, 'n_epochs': 46, 'lr_all': 0.012282954728762013, 'reg_all': 0.09992275508720494}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:27,637] Trial 981 finished with value: 0.9263012148489096 and parameters: {'algorithm': 'SVD', 'n_factors': 34, 'n_epochs': 49, 'lr_all': 0.02291450420107423, 'reg_all': 0.08028410008277821}. Best is trial 726 with value: 0.9

Done computing similarity matrix.
Estimating biases using als...


[I 2024-09-02 15:47:35,420] Trial 992 finished with value: 0.9259184340197685 and parameters: {'algorithm': 'SVD', 'n_factors': 26, 'n_epochs': 50, 'lr_all': 0.013180034416261616, 'reg_all': 0.07650224451045472}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:35,867] Trial 993 finished with value: 0.9227950319598682 and parameters: {'algorithm': 'SVD', 'n_factors': 30, 'n_epochs': 46, 'lr_all': 0.01923327966202463, 'reg_all': 0.09934170829033849}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:36,008] Trial 994 finished with value: 1.130325814702732 and parameters: {'algorithm': 'SVD', 'n_factors': 25, 'n_epochs': 5, 'lr_all': 0.00047868467336446716, 'reg_all': 0.08156625661961513}. Best is trial 726 with value: 0.9152262212548059.
[I 2024-09-02 15:47:36,482] Trial 995 finished with value: 0.9324232659880678 and parameters: {'algorithm': 'SVD', 'n_factors': 34, 'n_epochs': 48, 'lr_all': 0.01686911826429779, 'reg_all': 0.06374369410446915}. 

In [31]:
study.best_params

{'algorithm': 'SVD',
 'n_factors': 31,
 'n_epochs': 45,
 'lr_all': 0.013616794184574305,
 'reg_all': 0.09979059294555132}

In [32]:
best_params = study.best_params

best_algorithm = best_params['algorithm']

if best_algorithm == 'BaselineOnly':
    bsl_options = {
        'method': best_params['method'],
        'reg_u': best_params['reg_u'],
        'reg_i': best_params['reg_i']
    }
    final_algo = BaselineOnly(bsl_options=bsl_options)

elif best_algorithm == 'SVD':
    final_algo = SVD(
        n_factors=best_params['n_factors'],
        n_epochs=best_params['n_epochs'],
        lr_all=best_params['lr_all'],
        reg_all=best_params['reg_all']
    )

elif best_algorithm == 'KNNBasic':
    sim_options = {
        'name': best_params['name'],
        'user_based': best_params['user_based']
    }
    final_algo = KNNBasic(
        k=best_params['k'],
        min_k=best_params['min_k'],
        sim_options=sim_options
    )

elif best_algorithm == 'CoClustering':
    final_algo = CoClustering(
        n_cltr_u=best_params['n_cltr_u'],
        n_cltr_i=best_params['n_cltr_i'],
        n_epochs=best_params['n_epochs']
    )

final_algo.fit(trainset)

final_predictions = final_algo.test(testset)

print(f"mae = {accuracy.mae(final_predictions, verbose=False)}")
print(f"mse = {accuracy.mse(final_predictions, verbose=False)}")
print(f"rmse= {accuracy.rmse(final_predictions, verbose=False)}")
print(f"fcp = {accuracy.fcp(final_predictions, verbose=False)}")

mae = 0.69117038385164
mse = 0.8511011915170846
rmse= 0.9225514573816924
fcp = 0.6398318158635935
